In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pathlib
import PIL

from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from keras.applications.imagenet_utils import decode_predictions
from keras import applications
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [9]:
batch_size = 32
IMG_SIZE = 224
IMG_SIZE = 224

In [41]:
path = "data"
epochs = 10 


def generate_model():
    load_data(path)
    class_names, train_ds, val ds = train_val_split(path)
    base_model = define_model(len(class_names))
    train_model(epochs, base_model, train_ds, val_ds)
    save_model(base_model)

Successfully loaded 5620 images
Found 5616 files belonging to 8 classes.
Using 4493 files for training.
Found 5616 files belonging to 8 classes.
Using 1123 files for validation.
Dataset has 8 labels: ['airmax1', 'blazer', 'jordan11', 'jordan1_high', 'jordan4', 'yeezy350', 'yeezy700', 'yeezy_slide']


### 0. Load images from data folder

In [6]:
def load_data(path):
    data_dir = path
    data_dir = pathlib.Path(data_dir)
    image_count = len(list(data_dir.glob('*/*')))
    print(f"Successfully loaded {image_count} images")
    return data_dir

### 1. Split data into train and validation dataset

In [7]:
# 20% of images used for validation

def train_val_split(data_dir):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        labels = 'inferred',
        validation_split = 0.2,
        subset = "training",
        seed = 123,
        image_size = (img_height, img_width),
        batch_size = batch_size)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        labels = 'inferred',
        validation_split = 0.2,
        subset = "validation",
        seed = 123,
        image_size = (img_height, img_width),
        batch_size = batch_size
    )
    
    
    class_names = train_ds.class_names
    print()
    print(f"Dataset has {len(class_names)} labels: {class_names}")
    return class_names, train_ds, val_ds

### 2. Define model

In [6]:
# after multiple researches and test, ResNet 50 seems to be the best convolutional neural network

from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

def define_model(num_classes):
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

    # Create the base model from the pre-trained model MobileNet V2
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
    base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001), loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['acc'])
    return base_model


['jordan1_high', 'new_700', 'new_dunk', 'new_jordan4', 'yeezy350']


### 3. Train Model

In [7]:
def train_model(epochs, base_model, train_ds, val_ds):
    history = base_model.fit(
      train_ds,
      validation_data = val_ds,
      epochs = epochs
    )

Epoch 1/10


2022-02-11 17:08:29.404425: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-11 17:08:29.405359: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-11 17:08:29.811076: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


 7/85 [=>............................] - ETA: 44s - loss: 5.9100 - acc: 0.2500

2022-02-11 17:08:37.729592: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


38/85 [============>.................] - ETA: 26s - loss: 4.9924 - acc: 0.2837

...

68/85 [=======================>......] - ETA: 9s - loss: 0.1353 - acc: 0.9508

2022-02-11 17:16:59.937098: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


85/85 [==============================] - ETA: 0s - loss: 0.1307 - acc: 0.9517

2022-02-11 17:17:10.168872: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


85/85 [==============================] - 52s 602ms/step - loss: 0.1307 - acc: 0.9517 - val_loss: 1.1688 - val_acc: 0.7330


### 5. Save model

In [ ]:
def save_model(base_model):
    base_model.save_model("model/model.h5")
    
    # quantization used to allow a lighter model for mobile architectures
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    tflite_model_quant = converter.convert()
    # Save the model.
    with open('model/model.tflite', 'wb') as f:
      f.write(tflite_model)

### 6. External Tests

In [9]:
from PIL import Image, ImageOps
def some_test(model):
    labels = ["Yeezy Slide", "Blazer", "Jordan 11", "Air Max 1", "Yeezy 700", "Yeezy 350", "Jordan 4","Jordan 1 High"]
    for test in os.listdir("test_images"):
        try:
            data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
            image = Image.open('test_images/' + test)
            size = (224, 224)
            image = ImageOps.fit(image, size, Image.ANTIALIAS)
            image_array = np.asarray(image)
            normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
            data[0] = normalized_image_array
            prediction = model.predict(data)
            print(test)
            print(
                "This image most likely belongs to {} with a {:.2f} percent confidence."
                .format(labels[np.argmax(prediction)], 100 * np.max(prediction))
            )
        except:
            pass #got errors for .DS files

In [10]:
# main
if os.path.isfile('model/model_quantized.tflite') or os.path.isfile('model/model.h5'):
    print("Model already trained, loading weights..")
    model = tf.keras.models.load_model('model/model.h5', compile = False)
    print("Model successfully loaded!")
    print("Doing tests on test_images folder")
    some_test(model)
else:
    print("No model available, train a new one..")
    generate_model()

Model already trained, loading weights..


2022-02-18 16:22:03.254022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10.jpeg
This image most likely belongs to Jordan 1 High with a 99.99 percent confidence.
1.jpeg
This image most likely belongs to Yeezy Slide with a 98.18 percent confidence.
11.jpeg
This image most likely belongs to Blazer with a 99.96 percent confidence.
7.jpeg
This image most likely belongs to Jordan 1 High with a 99.99 percent confidence.
4.jpeg
This image most likely belongs to Yeezy 700 with a 100.00 percent confidence.
14.jpeg
This image most likely belongs to Air Max 1 with a 100.00 percent confidence.
15.jpeg
This image most likely belongs to Yeezy 700 with a 99.99 percent confidence.
9.jpeg
This image most likely belongs to Jordan 1 High with a 91.69 percent confidence.
2.jpeg
This image most likely belongs to Yeezy 350 with a 100.00 percent confidence.
12.jpeg
This image most likely belongs to Jordan 11 with a 100.00 percent confidence.
5.jpg
This image most likely belongs to Yeezy 700 with a 86.89 percent confidence.
13.jpeg
This image most likely belongs to Jordan 1 High w